<h2> Importing libaries </h2>

In [136]:
import requests
from No_sync.credentials import config 
import requests
from pprint import pprint
#from authentication import get_token
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

<h2> Handling API requests and data extraction</h2>

In [137]:
def get_token():
  """
  Get an access token from the API
  """

  if not config['client_id']:
    raise ValueError('client_id must be set in credentials.py')

  if not config['client_secret']:
    raise ValueError('client_secret must be set in credentials.py')

  req = requests.post(config['token_url'],
    data={
        'grant_type': 'client_credentials',
        'client_id': config['client_id'],
        'client_secret': config['client_secret'],
        'scope': 'api'
    },
    headers={'content-type': 'application/x-www-form-urlencoded'})

  req.raise_for_status()
  print('Token request successful')
  return req.json()

In [138]:
def get_week_summary(token, year, week):
  """
  Get the weekly summary for a given year and week
  """
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{year}/{week}"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()

In [139]:
def make_sample_request(token):
  """
  Make a sample request to the API
  """
  url = f"{config['api_base_url']}/v1/sample/auth"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()


In [140]:
#request a token
print(f"Requesting token from {config['token_url']}, using client_id {config['client_id']}.")
token = get_token()


Requesting token from https://id.barentswatch.no/connect/token, using client_id dinussen.sivarasalingam@nmbu.no:Fishdata.


Token request successful


In [141]:
# test token
response = make_sample_request(token)
print('Request to the api was successful - authentication worked.')
pprint(response)

Request to the api was successful - authentication worked.
{'id': 1879182644,
 'name': 'https://www.barentswatch.no',
 'updated': '2023-10-02T13:48:27.0597974+02:00'}


<h2> Data transformation </h2>
<h3> Getting year worth of data </h3>

In [142]:
import pandas as pd
from datetime import datetime, timedelta

def weeks_of_the_year():
    """
    Create a DataFrame with the week number and year for each week of the year
    """
    # Get the current year and the week of the year
    today = datetime.now()
    year = today.year
    day_of_year = today.strftime('%j')
    week_of_year = (int(day_of_year) - 1) // 7 + 1
    
    # Calculate the number of weeks left in the previous year
    weeks_left_last_year = 52 - week_of_year
    
    # Create DataFrames for the current year and the remaining weeks from the previous year
    week_df = pd.DataFrame({'Week': range(1, week_of_year + 1), 'Year': [year] * week_of_year})
    week_df_last = pd.DataFrame({'Week': range(1, weeks_left_last_year + 1), 'Year': [year - 1] * weeks_left_last_year})

    # Concatenate the two DataFrames
    week_df = pd.concat([week_df_last, week_df], ignore_index=True)
    # add week_of_year to week where year is current year - 1
    week_df.loc[week_df['Year'] == year - 1, 'Week'] += week_of_year

    return week_df

# Example usage:
weeks_df = weeks_of_the_year()

In [143]:
def make_df(weeksummary):
    """
    Create a DataFrame from the week summary
    """
    df_data = pd.DataFrame()
    #print(weeksummary)
    for element in weeksummary:
        #print(element)
        if element == 'localities':
            lines = []
            for information in weeksummary['localities']:
                line = []
                for key, value in information.items():
                    #print(key, value)
                    df_data[key] = value
                    line.append(value)
                df_data['year'] = (weeksummary['year'])
                line.append(weeksummary['year'])
                df_data['week'] = weeksummary['week']
                line.append(weeksummary['week'])
                #print(line)
            
                #add line to df_data as row
                lines.append(line)
    
    df_data = pd.DataFrame(lines, columns = df_data.columns)
    #print(df_data)
    return df_data

In [144]:
def get_year_data(weeks_df):
    """
    This function returns a dataframe with all the data from the weeks in weeks_df
    """
    teller = 0
    df_data = pd.DataFrame()
    for week, year in weeks_df.values:
        teller +=1
        #print(week, year)
        weeksummary= get_week_summary(token,year,week)
        #print(weeksummary)
        df_to_concat = make_df(weeksummary)
        #print(df_to_concat)
        frames = [df_data, df_to_concat]
        df_data = pd.concat(frames, ignore_index=True)
        #print(teller)
    return df_data

In [145]:
summary_oneyear = get_year_data(weeks_df)
summary_oneyear.shape

year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
localities
year
week
loc

(89782, 21)

In [146]:
summary_oneyear.to_csv('Data/summary_oneyear.csv', index=False)


In [147]:
# # find null values for each column
# summary_oneyear.isnull().sum()

# #remove avgAdultFemaleLice column
# summary_oneyear = summary_oneyear.drop(columns=['avgAdultFemaleLice'])

<h3> Get lice count from one locality </h3>

In [131]:
def get_detailed_week(token, year, week, localityId):
  """
  Get the weekly summary for a given year and week
  """
  url = f"{config['api_base_url']}/v1/geodata/fishhealth/locality/{localityId}/{year}/{week}"
  headers ={
    'authorization': 'Bearer ' + token['access_token'],
    'content-type': 'application/json',
  }

  response = requests.get(url, headers=headers)
  response.raise_for_status()
  return response.json()

In [170]:
def extract_columns_from_json(json_data, desired_columns=['avgAdultFemaleLice', 'avgMobileLice', 'avgStationaryLice', 'localityName', 'speciesList']):
    """
    Extract specified columns from a JSON input and create a DataFrame.

    :param json_data: JSON data as a dictionary.
    :param desired_columns: List of column names to extract.
    :return: A DataFrame containing the extracted columns.
    """
    # Extract columns from 'localityWeek' if they exist
    locality_week_data = json_data.get('localityWeek', {})
    locality_week_columns = {col: locality_week_data.get(col) for col in ['avgAdultFemaleLice', 'avgMobileLice', 'avgStationaryLice']}
    
    # Extract 'speciesList' from 'aquaCultureRegister' if it exists
    aqua_culture_register_data = json_data.get('aquaCultureRegister', {})
    species_list = aqua_culture_register_data.get('speciesList')

    # Combine the extracted columns into a single dictionary
    filtered_data = {
        'avgAdultFemaleLice': locality_week_columns.get('avgAdultFemaleLice'),
        'avgMobileLice': locality_week_columns.get('avgMobileLice'),
        'avgStationaryLice': locality_week_columns.get('avgStationaryLice'),
        'localityName': json_data.get('localityName'),
        'speciesList': species_list
    }

    # Convert the dictionary into a DataFrame
    df = pd.DataFrame([filtered_data])
    
    return df


In [171]:

import pandas as pd

def get_lice_counts(token, weeks_df, locality=32277):

    lice_counts = pd.DataFrame()
    for week, year in weeks_df.values:
        weekdetails = get_detailed_week(token, year, week, locality)

        # Extract the desired columns from the JSON data
        df = extract_columns_from_json(weekdetails)

        # Add the year and week columns
        df['year'] = year
        df['week'] = week

        # Concatenate the DataFrames
        lice_counts = pd.concat([lice_counts, df], ignore_index=True)
    return lice_counts
lice_counts = get_lice_counts(token, weeks_df)




In [173]:
lice_counts.to_csv('Data/lice_counts.csv', index=False)

In [161]:
result_df

,avgAdultFemaleLice,avgMobileLice,avgStationaryLice,localityName,speciesList
0,None,None,None,Nmbu Fiskelaboratoriet,None


<h2> Connecting to Cassandra cluster and data loading</h2>

In [116]:
# Connecting to Cassandra
from cassandra.cluster import Cluster
cluster = Cluster(['localhost'], port=9042)
session = cluster.connect()

In [117]:
# Creating a keyspace
session.execute("CREATE KEYSPACE IF NOT EXISTS fish_keyspace WITH REPLICATION = { 'class' : 'SimpleStrategy', 'replication_factor' : 1 };")

In [118]:
# Create a new table (first time only)
session.set_keyspace('fish_keyspace')

# Drop the table if it already exists
session.execute("DROP TABLE IF EXISTS fish_keyspace.fish_table_year;") 

# Create a new table
session.execute("""
    CREATE TABLE IF NOT EXISTS fish_table_year (
        localityNo INT,
        localityweekid INT PRIMARY KEY,
        name TEXT,
        hasReportedLice BOOLEAN,
        isFallow BOOLEAN,
        avgAdultFemaleLice DOUBLE,
        hasCleanerfishDeployed BOOLEAN,
        hasMechanicalRemoval BOOLEAN,
        hasSubstanceTreatments BOOLEAN,
        hasPd BOOLEAN,
        hasIla BOOLEAN,
        municipalityNo INT,
        municipality TEXT,
        lat DOUBLE,
        lon DOUBLE,
        isOnLand BOOLEAN,
        inFilteredSelection BOOLEAN,
        hasSalmonoids BOOLEAN,
        isSlaughterHoldingCage BOOLEAN,
        year INT,
        week INT
    );
""")


# session.execute("CREATE TABLE IF NOT EXISTS fish_table_year (localityNo INT, localityWeekId INT, year INT, week INT, PRIMARY KEY ((localityNo, localityWeekId, year, week)), name TEXT, hasReportedLice BOOLEAN, isFallow BOOLEAN, avgAdultFemaleLice DOUBLE, hasCleanerfishDeployed BOOLEAN, hasMechanicalRemoval BOOLEAN, hasSubstanceTreatments BOOLEAN, hasPd BOOLEAN, hasIla BOOLEAN, municipalityNo INT, municipality TEXT, lat DOUBLE, lon DOUBLE, isOnLand BOOLEAN, inFilteredSelection BOOLEAN, hasSalmonoids BOOLEAN, isSlaughterHoldingCage BOOLEAN);")




In [119]:
import os
os.environ["JAVA_HOME"] = "C:\Program Files\Java\jdk-20"
# If you are using environments in Python, you can set the environment variables like this:
os.environ["PYSPARK_PYTHON"] = "python" # or similar to "/Users/kristian/miniforge3/envs/tf_M1/bin/python"
#os.environ["PYSPARK_DRIVER_PYTHON"] = "python" # or similar to "/Users/kristian/miniforge3/envs/tf_M1/bin/python"
# Set the Hadoop version to the one you are using, e.g., none:
#os.environ["PYSPARK_HADOOP_VERSION"] = "without"

In [120]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('SparkCassandraApp').\
    config('spark.jars.packages', 'com.datastax.spark:spark-cassandra-connector_2.12:3.4.1').\
    config('spark.cassandra.connection.host', 'localhost').\
    config('spark.sql.extensions', 'com.datastax.spark.connector.CassandraSparkExtensions').\
    config('spark.sql.catalog.mycatalog', 'com.datastax.spark.connector.datasource.CassandraCatalog').\
    config('spark.cassandra.connection.port', '9042').getOrCreate()
# Some warnings are to be expected.

In [121]:
YearData = spark.read.csv('Data/summary_oneyear.csv', header=True, inferSchema=True)


#show types
YearData.printSchema()


root
 |-- localityNo: integer (nullable = true)
 |-- localityWeekId: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- hasReportedLice: boolean (nullable = true)
 |-- isFallow: boolean (nullable = true)
 |-- avgAdultFemaleLice: double (nullable = true)
 |-- hasCleanerfishDeployed: boolean (nullable = true)
 |-- hasMechanicalRemoval: boolean (nullable = true)
 |-- hasSubstanceTreatments: boolean (nullable = true)
 |-- hasPd: boolean (nullable = true)
 |-- hasIla: boolean (nullable = true)
 |-- municipalityNo: integer (nullable = true)
 |-- municipality: string (nullable = true)
 |-- lat: double (nullable = true)
 |-- lon: double (nullable = true)
 |-- isOnLand: boolean (nullable = true)
 |-- inFilteredSelection: boolean (nullable = true)
 |-- hasSalmonoids: boolean (nullable = true)
 |-- isSlaughterHoldingCage: boolean (nullable = true)
 |-- year: integer (nullable = true)
 |-- week: integer (nullable = true)



In [122]:
keyspace = "fish_keyspace"
table = "fish_table_year"

YearData.write \
    .format("org.apache.spark.sql.cassandra") \
    .option("table", table) \
    .option("keyspace", keyspace) \
    .mode("append").save()
    


IllegalArgumentException: Attempting to write to C* Table but missing
primary key columns: [localityweekid]